# SQL Project
You were hired by Ironhack to perform an Analytics Consulting Project entitled: competitive landscape.

Your mission is to create and populate an appropriate database with many coding schools that are our competition, as well as design an suitable queries that answer business questions of interest (to be defined by you)


**Suggested Steps in the Project:**


*   Read this notebook and understand each function. Comment the code appropriately

*   Populate the list of schools with a wider variety of schools (how are you going to get the school ID?)

* Take a look at the obtained dataframes. What dimensions do you have? what keys do you have? how could the different dataframes be connected?

* Go back to the drawing board and try to create an entity relationship diagram for tables available

* Once you have the schemas you want, you will need to:
  - create the suitable SQL queries to create the tables and populate them
  - run these queries using the appropriate Python connectors
  
* Bonus: How will this datamodel be updated in the future? Please write auxiliary functions that test the database for data quality issues. For example: how could you make sure you only include the most recent comments when you re-run the script?


# Suggested Deliverables

* 5-6 minute presentation of data model created, decision process and business analysis proposed

* exported .sql file with the final schema

* Supporting python files used to generate all logic

* High level documentation explaining tables designed and focusing on update methods

Crucial hint: check out the following tutorial:
https://www.dataquest.io/blog/sql-insert-tutorial/


In [9]:
# you must populate this dict with the schools required -> try talking to the teaching team about this

# To get the School ID, we went to the reviews list of the school, then using INSPECTOR tool, we looked at
# the network request that is being made when you press the pagination buttons at the end of the review list.
# The URL used to fetch data there has school name and ID as parameter

schools = {   
    'ironhack' : 10828,
    'app-academy' : 10525,
    'springboard' : 11035,
    'actualize': 10505,
    'learningfuze': 10862,
    'clarusway': 11539,
    'codesmith':10643,
    'code-platoon':10627,
    'knowledgehut':10846,
    'udacity':11118,
    'altcademy':10517,
    'tech-elevator':11056,
    'codeworks': 10650,
    '4geeks-academy': 10492,
    'coding-temple': 10664,
    'devcodecamp': 10703,
    'nucamp': 10923,
    'tripleten': 11225,
    'skillcrush':11020,
    'the-tech-academy': 11091,
    'hack-reactor': 10788,
    'tech-i-s': 11283,
    'brainstation': 10571,
    'coding-dojo': 10659
}

In [11]:
import re
import pandas as pd
from pandas.io.json import json_normalize
import requests



def get_comments_school(school):
    print(school)
    TAG_RE = re.compile(r'<[^>]+>')
    # defines url to make api call to data -> dynamic with school if you want to scrape competition
    url = "https://www.switchup.org/chimera/v1/school-review-list?mainTemplate=school-review-list&path=%2Fbootcamps%2F" + school + "&isDataTarget=false&page=3&perPage=10000&simpleHtml=true&truncationLength=250"
    #makes get request and converts answer to json
    # url defines the page of all the information, request is made, and information is returned to data variable
    data = requests.get(url).json()
    #converts json to dataframe
    reviews =  pd.DataFrame(data['content']['reviews'])
  
    #aux function to apply regex and remove tags
    def remove_tags(x):
        return TAG_RE.sub('',x)
    reviews['review_body'] = reviews['body'].apply(remove_tags)
    reviews['school'] = school
    return reviews

In [12]:
# could you write this as a list comprehension? ;)
comments =pd.concat([get_comments_school(school) for school in schools.keys()])

ironhack
app-academy
springboard
actualize
learningfuze
clarusway
codesmith
code-platoon
knowledgehut
udacity
altcademy
tech-elevator
codeworks
4geeks-academy
coding-temple
devcodecamp
nucamp
tripleten
skillcrush
the-tech-academy
hack-reactor
tech-i-s
brainstation
coding-dojo


In [15]:
comments.info()
comments.to_csv('comments.csv')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10716 entries, 0 to 489
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               10716 non-null  int64  
 1   name             10716 non-null  object 
 2   anonymous        10716 non-null  bool   
 3   hostProgramName  3999 non-null   object 
 4   graduatingYear   10663 non-null  float64
 5   isAlumni         10715 non-null  object 
 6   jobTitle         6624 non-null   object 
 7   tagline          10715 non-null  object 
 8   body             10716 non-null  object 
 9   rawBody          10716 non-null  object 
 10  createdAt        10716 non-null  object 
 11  queryDate        10716 non-null  object 
 12  program          9396 non-null   object 
 13  user             10716 non-null  object 
 14  overallScore     10703 non-null  object 
 15  comments         10716 non-null  object 
 16  overall          10701 non-null  object 
 17  curriculum    

In [17]:
from pandas import json_normalize

def get_school_info(school, school_id):
    url = 'https://www.switchup.org/chimera/v1/bootcamp-data?mainTemplate=bootcamp-data%2Fdescription&path=%2Fbootcamps%2F'+ str(school) + '&isDataTarget=false&bootcampId='+ str(school_id) + '&logoTag=logo&truncationLength=250&readMoreOmission=...&readMoreText=Read%20More&readLessText=Read%20Less'

    data = requests.get(url).json()

    data.keys()

    courses = data['content']['courses']
    courses_df = pd.DataFrame(courses, columns= ['courses'])

    locations = data['content']['locations']
    locations_df = json_normalize(locations)

    badges_df = pd.DataFrame(data['content']['meritBadges'])
    
    website = data['content']['webaddr']
    description = data['content']['description']
    logoUrl = data['content']['logoUrl']
    school_df = pd.DataFrame([website,description,logoUrl]).T
    school_df.columns =  ['website','description','LogoUrl']

    locations_df['school'] = school
    courses_df['school'] = school
    badges_df['school'] = school
    school_df['school'] = school
    

    locations_df['school_id'] = school_id
    courses_df['school_id'] = school_id
    badges_df['school_id'] = school_id
    school_df['school_id'] = school_id

    return locations_df, courses_df, badges_df, school_df

locations_list = []
courses_list = []
badges_list = []
schools_list = []

for school, id in schools.items():
    print(school)
    a,b,c,d = get_school_info(school,id)
    
    locations_list.append(a)
    courses_list.append(b)
    badges_list.append(c)
    schools_list.append(d)

ironhack
app-academy
springboard
actualize
learningfuze
clarusway
codesmith
code-platoon
knowledgehut
udacity
altcademy
tech-elevator
codeworks
4geeks-academy
coding-temple
devcodecamp
nucamp
tripleten
skillcrush
the-tech-academy
hack-reactor
tech-i-s
brainstation
coding-dojo


In [18]:
locations_list

[      id               description  country.id   country.name country.abbrev  \
 0  15901           Berlin, Germany        57.0        Germany             DE   
 1  16022       Mexico City, Mexico        29.0         Mexico             MX   
 2  16086    Amsterdam, Netherlands        59.0    Netherlands             NL   
 3  16088         Sao Paulo, Brazil        42.0         Brazil             BR   
 4  16109             Paris, France        38.0         France             FR   
 5  16375  Miami, FL, United States         1.0  United States             US   
 6  16376             Madrid, Spain        12.0          Spain             ES   
 7  16377          Barcelona, Spain        12.0          Spain             ES   
 8  16709          Lisbon, Portugal        28.0       Portugal             PT   
 9  17233                    Online         NaN            NaN            NaN   
 
    city.id    city.name city.keyword  state.id state.name state.abbrev  \
 0  31156.0       Berlin       b

In [20]:
locations = pd.concat(locations_list)
locations.to_csv('locations.csv')
locations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 195 entries, 0 to 7
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              195 non-null    int64  
 1   description     195 non-null    object 
 2   country.id      171 non-null    float64
 3   country.name    171 non-null    object 
 4   country.abbrev  171 non-null    object 
 5   city.id         171 non-null    float64
 6   city.name       171 non-null    object 
 7   city.keyword    171 non-null    object 
 8   state.id        158 non-null    float64
 9   state.name      158 non-null    object 
 10  state.abbrev    158 non-null    object 
 11  state.keyword   158 non-null    object 
 12  school          195 non-null    object 
 13  school_id       195 non-null    int64  
dtypes: float64(3), int64(2), object(9)
memory usage: 22.9+ KB


In [21]:
courses = pd.concat(courses_list)
courses.to_csv('courses.csv')
courses.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 216 entries, 0 to 8
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   courses    216 non-null    object
 1   school     216 non-null    object
 2   school_id  216 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 6.8+ KB


In [22]:
badges = pd.concat(badges_list)
badges.to_csv('badges.csv')
badges.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68 entries, 0 to 3
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         68 non-null     object
 1   keyword      68 non-null     object
 2   description  68 non-null     object
 3   school       68 non-null     object
 4   school_id    68 non-null     int64 
dtypes: int64(1), object(4)
memory usage: 3.2+ KB


In [23]:
# any data cleaning still missing here? take a look at the description
schools = pd.concat(schools_list)
schools.to_csv('schools.csv')
schools.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24 entries, 0 to 0
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   website      24 non-null     object
 1   description  24 non-null     object
 2   LogoUrl      24 non-null     object
 3   school       24 non-null     object
 4   school_id    24 non-null     int64 
dtypes: int64(1), object(4)
memory usage: 1.1+ KB
